![MLU Logo](../images/MLU_Logo.png)

# MLU-NLP2 Final Project

In [2]:
##torchtext is a package within pytorch consisting of data processing utilities and popular datasets for natural language
!pip install -q torchtext torch
# !pip install -qU regex requests hydra-core omegaconf
# !pip install -qU cython
!pip install -q transformers
!pip install -q fsspec
!pip install -q datasets
!pip install -q torch torchtext

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
import boto3
import os
import numpy as np
import torch
from torch import nn
from sklearn.metrics import f1_score
from tqdm import tqdm, tqdm_notebook
import torchtext
from nltk import word_tokenize
import random
from torch import optim

from pprint import pprint as pprint

### Reading the dataset
The datasets are in our MLU datalake and can be downloaded to your local instance here

In [2]:
# import the datasets
bucketname = 'mlu-courses-datalake' 
s3 = boto3.resource('s3')

s3.Bucket(bucketname).download_file('NLP2/data/training.csv', 
                                         './training.csv')
s3.Bucket(bucketname).download_file('NLP2/data/public_test_features.csv', 
                                         './public_test_features.csv')
s3.Bucket(bucketname).download_file('NLP2/data/glove.6B.100d.txt', 
                                         './glove.6B.100d.txt')

In [3]:
TRAIN_DATA_FILE ='./training.csv'
TEST_DATA_FILE = './public_test_features.csv'
GLOVE_DATA_FILE = './glove.6B.100d.txt'

In [4]:
# Organize inputs into Datasets
from datasets import load_dataset
train_dataset_original, val_dataset_original = load_dataset('csv', 
                                                            data_files=[TRAIN_DATA_FILE], 
                                                            split=['train[:85%]', 'train[85%:]'])
test_dataset_original = load_dataset('csv', data_files=[TEST_DATA_FILE], split='train')

Using custom data configuration default-004aa1dfe52d198e


0 tables [00:00, ? tables/s]

Using custom data configuration default-111060e8804a730b


Dataset csv downloaded and prepared to /home/ec2-user/.cache/huggingface/datasets/csv/default-004aa1dfe52d198e/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.


0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /home/ec2-user/.cache/huggingface/datasets/csv/default-111060e8804a730b/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.


Below, we are combining question and answer in each row as 1 single text column for simplicity. Alternatively, we can run two parallel networks for question and answer, merge the output of the 2 networks and have a classification layer as output. You may choose to save the files for ease of use, in future steps.

## Data Preparation
Tokenize data and put into DataLoaders.

In [ ]:
# Currently using huggingface API. Most pretrained LMs at https://huggingface.co/models can be subbed out with
# only minor modifications
# Initially tried using RoBERTa, got ~0.6 F1 score on val set after adjusting hyperparams.
# Best submissions were with BERT-large, previously fine-tuned on SQuAD dataset. Got ~0.7 F1 score on val sets. 
PRETRAINED_MODEL = 'bert-large-uncased-whole-word-masking-finetuned-squad'

In [5]:
from datasets import Value

# RoBERTa uses byte-pair encoding tokenizer, accounts for unicode/special chars
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)

def tokenize_qa(dataset):
    """Tokenize question + answer with <sep> token between."""
    return tokenizer(dataset['question'], dataset['answer'], padding='max_length', truncation=True)

train_dataset = train_dataset_original.map(tokenize_qa, batched="True")
val_dataset = val_dataset_original.map(tokenize_qa, batched="True")
test_dataset = test_dataset_original.map(tokenize_qa, batched="True")

train_dataset = train_dataset.remove_columns(['question', 'answer', 'ID'])
val_dataset = val_dataset.remove_columns(['question', 'answer', 'ID'])
test_dataset = test_dataset.remove_columns(['question', 'answer'])

train_dataset = train_dataset.rename_column('relevance', 'labels')
val_dataset = val_dataset.rename_column('relevance', 'labels')

train_features = train_dataset.features.copy()
train_features['labels'] = Value("float32")
train_dataset = train_dataset.cast(train_features)
val_dataset = val_dataset.cast(train_features)

train_dataset.set_format('torch')
val_dataset.set_format('torch')
test_dataset.set_format('torch')

mini_train_dataset = train_dataset.shuffle(seed=1234).select(range(300))

print(train_dataset, val_dataset, test_dataset)
print(mini_train_dataset)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['attention_mask', 'input_ids', 'labels', 'token_type_ids'],
    num_rows: 6792
}) Dataset({
    features: ['attention_mask', 'input_ids', 'labels', 'token_type_ids'],
    num_rows: 69
}) Dataset({
    features: ['ID', 'attention_mask', 'input_ids', 'token_type_ids'],
    num_rows: 2941
})
Dataset({
    features: ['attention_mask', 'input_ids', 'labels', 'token_type_ids'],
    num_rows: 300
})


In [6]:
from torch.utils.data import DataLoader

BATCH_SIZE = 16

mini_train_dataloader = DataLoader(mini_train_dataset, shuffle=True, batch_size=BATCH_SIZE)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=BATCH_SIZE)
val_dataloader = DataLoader(val_dataset, shuffle=True, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

## Define Model

Load a pre-trained RoBERTa model for binary classification and fine-tune on our task.

In [7]:
# Only run if out of GPU space. Sometimes can help.
del model
torch.cuda.empty_cache()

NameError: name 'model' is not defined

In [7]:
from transformers import AutoModelForSequenceClassification

# Download pretrained model for sequence classification. 
# Pool outputs from transformer and run through one-layer NN with dropout.
model = AutoModelForSequenceClassification.from_pretrained(PRETRAINED_MODEL, num_labels=1)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs.")
    model = nn.DataParallel(model)

model.to(device)

Downloading:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForSequenceClassification: ['qa_outputs.bias', 'qa_outputs.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predict

Using 8 GPUs.


DataParallel(
  (module): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 1024, padding_idx=0)
        (position_embeddings): Embedding(512, 1024)
        (token_type_embeddings): Embedding(2, 1024)
        (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=1024, out_features=1024, bias=True)
                (key): Linear(in_features=1024, out_features=1024, bias=True)
                (value): Linear(in_features=1024, out_features=1024, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=1024, out_features=10

### Training

In [8]:
train_dl = train_dataloader
# train_dl = mini_train_dataloader
val_dl = val_dataloader
test_dl = test_dataloader

In [9]:
print(device)

from transformers import AdamW
from transformers import get_scheduler

# Start with training hyperparams from https://arxiv.org/pdf/1905.05583v3.pdf
optimizer = AdamW(model.parameters(), lr=3e-5)

num_epochs = 2
num_training_steps = num_epochs * len(train_dl)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

cuda


In [10]:
# Train model
from tqdm.auto import tqdm
progress_bar = tqdm(range(num_training_steps * train_dl.batch_size))

# Additional loss penalty for positive examples, ended up unnecessary on most models. 
POS_LOSS_WEIGHT = 1

# Override loss fn for BCE rather than using default in model
loss_fn = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([POS_LOSS_WEIGHT]).to(device))

model.train()
for epoch in range(num_epochs):
    running_losses = []
    for batch in train_dl:
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        
        logits = outputs.logits.view(-1)
        loss = loss_fn(logits, batch['labels']).mean()
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(batch['attention_mask'].shape[0])
        running_losses.append(loss.sum())
        progress_bar.set_description(f'{sum(running_losses) / len(running_losses)}', refresh=True)

  0%|          | 0/13600 [00:00<?, ?it/s]

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [11]:
import numpy as np
from datasets import load_metric

progress_bar = tqdm(range(len(val_dl)))

yhats, labels = [], []

# Collect sigmoid outputs directly to compare different cutoffs
model.eval()
for batch in val_dl:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        logits = model(**batch).logits.view(-1)
    
    yhat_batch = torch.sigmoid(logits).cpu().data.numpy().tolist()
    yhats.extend(yhat_batch)
    labels.extend(batch['labels'])
    
    progress_bar.update(1)

#     preds = torch.sigmoid(yhat) > CUTOFF
#     print(preds)
#     print(batch['labels'])
#     predictions = torch.argmax(logits, dim=-1)



  0%|          | 0/5 [00:00<?, ?it/s]

In [12]:
# Determine best cutoff to maximize f1 on val
# (maybe can use binary search to balance precision/recall but seems overly specific on val)
best_cutoff = None
best_metrics = None
best_metrics_scores = None
for cutoff in tqdm(range(0, 100, 5)):
    cutoff = cutoff / 100

    metrics = []
    metrics.append(load_metric("f1"))
    metrics.append(load_metric("accuracy"))
    metrics.append(load_metric("precision"))
    metrics.append(load_metric("recall"))

    preds = [yhat > cutoff for yhat in yhats]
    for metric in metrics:
        metric.add_batch(predictions=preds, references=labels)

    metrics_scores = [metric.compute() for metric in metrics]
#     print(f'{cutoff}:\n {metrics_scores}')
    
    if best_metrics_scores is None or metrics_scores[0]['f1'] > best_metrics_scores[0]['f1']:
        best_cutoff = cutoff
        best_metrics = metrics
        best_metrics_scores = metrics_scores
    
print(f"\nBest cutoff at {best_cutoff}:")
print(''.join([str(score) for score in best_metrics_scores]))

  0%|          | 0/20 [00:00<?, ?it/s]

0.0:
 [{'f1': 0.23076923076923078}, {'accuracy': 0.13043478260869565}, {'precision': 0.13043478260869565}, {'recall': 1.0}]
0.05:
 [{'f1': 0.782608695652174}, {'accuracy': 0.927536231884058}, {'precision': 0.6428571428571429}, {'recall': 1.0}]
0.1:
 [{'f1': 0.761904761904762}, {'accuracy': 0.927536231884058}, {'precision': 0.6666666666666666}, {'recall': 0.8888888888888888}]
0.15:
 [{'f1': 0.761904761904762}, {'accuracy': 0.927536231884058}, {'precision': 0.6666666666666666}, {'recall': 0.8888888888888888}]
0.2:
 [{'f1': 0.8421052631578948}, {'accuracy': 0.9565217391304348}, {'precision': 0.8}, {'recall': 0.8888888888888888}]
0.25:
 [{'f1': 0.75}, {'accuracy': 0.9420289855072463}, {'precision': 0.8571428571428571}, {'recall': 0.6666666666666666}]
0.3:
 [{'f1': 0.75}, {'accuracy': 0.9420289855072463}, {'precision': 0.8571428571428571}, {'recall': 0.6666666666666666}]
0.35:
 [{'f1': 0.8}, {'accuracy': 0.9565217391304348}, {'precision': 1.0}, {'recall': 0.6666666666666666}]
0.4:
 [{'f1': 

In [13]:
# (Used when training on full dataset as there's no val set to tune cutoff)
DEFAULT_CUTOFF = 0.65

### Prediction

Below function is used to predict on test dataset using trained model. It returns a list of predicted probabilities

In [14]:
# Make prediction on test dataset
result_df = pd.DataFrame(columns=["ID", "relevance"])

model.eval()
for batch in tqdm(test_dl):
    ids = batch['ID']
    batch = {k: v.to(device) for k, v in batch.items() if k != 'ID'}
    with torch.no_grad():
        yhat = model(**batch).logits.view(-1)
        
    preds = torch.sigmoid(yhat) > best_cutoff

    for ID, pred in zip(ids, preds):
        result_df = result_df.append({'ID': int(ID), 'relevance': int(pred)}, ignore_index=True)
    
print(result_df)

  0%|          | 0/184 [00:00<?, ?it/s]

        ID relevance
0      917         0
1     6587         0
2     5227         0
3     4707         0
4      700         0
...    ...       ...
2936  5590         0
2937  5320         0
2938  1664         0
2939  1245         0
2940  4278         0

[2941 rows x 2 columns]


In [15]:
num_positives = result_df.relevance.sum()
print(f"Test predictions {num_positives / len(result_df)}% positives")

Test predictions 0.11730703842230533% positives


### Submit Results

Create a new dataframe for submission. The list of predicted probabilities are converted to labels using the pre-defined threshold of 0.15 (can be tuned for better performance). The list of labels is concatenated with the original sequential ID from the test file downloaded from Leaderboard, to generate the final submission

For submission, follow these steps:
1. Go to the folder where your notebook is in Sagemaker
2. Donwload the file __test_submission_nlp2.csv__ to your local machine
3. On NLP2 Leaderboard contest, select option __My Submissions"__ and upload your file

In [17]:
result_df.to_csv("test_submission_nlp2.csv", index=False)

### Model details for submission paste
Fine-tuned BERT-large with whole-word masking, previously fine-tuned on SQuAD QA. (see https://huggingface.co/bert-large-uncased-whole-word-masking-finetuned-squad)
BCE with no additional positive weighting, 0.5 cutoff. Trained on full train dataset.